In [ ]:
# Vehicle Type Classification Project

## Step 1: Import Required Libraries
import numpy as np
import pandas as pd
import os
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import matplotlib.pyplot as plt



In [ ]:

## Step 2: Load Dataset
# Update the dataset path to your environment
DATASET_PATH = "../Project/archive/Dataset"

# Load images using ImageDataGenerator
image_gen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_data = image_gen.flow_from_directory(
    DATASET_PATH,
    target_size=(128, 128),
    batch_size=32,
    class_mode='sparse',
    subset='training'
)

validation_data = image_gen.flow_from_directory(
    DATASET_PATH,
    target_size=(128, 128),
    batch_size=32,
    class_mode='sparse',
    subset='validation'
)

# Retrieve class labels
labels = list(train_data.class_indices.keys())


In [ ]:
## Step 3: Train a Conventional Model
# Flatten images for Random Forest Classifier
X_train_flat = train_data[0][0].reshape(train_data[0][0].shape[0], -1)
y_train = train_data[0][1]

X_test_flat = validation_data[0][0].reshape(validation_data[0][0].shape[0], -1)
y_test = validation_data[0][1]

clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train_flat, y_train)

# Evaluate the Random Forest Model
preds = clf.predict(X_test_flat)
rf_accuracy = accuracy_score(y_test, preds)
print("Random Forest Classifier Accuracy:", rf_accuracy)
print("\nClassification Report:\n", classification_report(y_test, preds, target_names=labels))


In [ ]:
## Step 4: Train a CNN Model
cnn_model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(len(labels), activation='softmax')
])

cnn_model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

history = cnn_model.fit(
    train_data,
    validation_data=validation_data,
    epochs=10
)


In [ ]:
## Step 5: Evaluate CNN Model
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 4))
plt.subplot(1, 2, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

# Save the trained CNN model
cnn_model.save("vehicle_classification_model.h5")

# Print final accuracies
final_train_accuracy = acc[-1]
final_val_accuracy = val_acc[-1]
print(f"Final CNN Training Accuracy: {final_train_accuracy:.2f}")
print(f"Final CNN Validation Accuracy: {final_val_accuracy:.2f}")
